I was not 100% sure, but most GO enrichment methods seemed to focus on DEG data, so I have essentially just taken all GO terms and their frequency over the entire genome and compare them at the TRD loci.

In [1]:
GFF=ape::read.gff("../Shiny/data/sace_R64-3-1_20210421.gff.gz")

In [2]:
library(data.table)
library(tidytable)
library(ggplot2)
    library(rrvgo)
source("~/BrusselSprouts/scripts/functions.R")

Warning message:
"package 'data.table' was built under R version 4.1.3"
Warning message:
"package 'tidytable' was built under R version 4.1.3"

Attaching package: 'tidytable'


The following objects are masked from 'package:data.table':

    between, first, fread, last


The following objects are masked from 'package:stats':

    dt, filter, lag


The following object is masked from 'package:base':

    %in%


Warning message:
"package 'ggplot2' was built under R version 4.1.3"


Warning message:
"package 'dplyr' was built under R version 4.1.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:tidytable':

    across, add_count, add_tally, anti_join, arrange, between,
    bind_cols, bind_rows, c_across, case_match, case_when, coalesce,
    consecutive_id, count, cross_join, cume_dist, cur_column, cur_data,
    cur_group_id, cur_group_rows, dense_rank, desc, distinct, filter,
    first, full_join, group_by, group_cols, group_split, group_vars,
    if_all, if_

In [3]:
output_dir<-"/home/jnrunge/data/trd/GO_data"
if(!dir.exists(output_dir)){
    dir.create(output_dir)
}

In [4]:
# load all TRD regions
TRD_regions<-fread_and_bind_files(list.files("/home/jnrunge/data/TRD/results/shiny/",pattern="TRD_regions.csv.gz$",full.names = TRUE))
TRD_regions<-mutate(TRD_regions, file=basename(file))%>%separate(file, c("cross","discard"), "-")%>%select(-discard)%>%filter(chr_start==chr_end)%>%
rename(chr=chr_start)%>%select(-chr_end)
# translate global positions to chr positions

for(c in select(TRD_regions,cross)%>%distinct()%>%pull())
    {
    AF<-fread(paste0("/home/jnrunge/data/TRD/results/shiny/",c,"-AF.csv.gz"))
    
    tmp_TRD_regions<-filter(TRD_regions, cross==c)%>%select(-any_of("start"))%>%
              left_join(select(AF, chr,pos,global_pos)%>%
              rename(global_start=global_pos), by=c("chr","global_start"))%>%rename(start=pos)
    
    TRD_regions<-filter(TRD_regions, cross!=c)%>%bind_rows(tmp_TRD_regions)
}

TRD_regions<-mutate(TRD_regions, end=global_end-(global_start-start))

head(TRD_regions)

ID,lengthSNPs,chr,global_start,global_end,lengthBp,cross,start,end
<int>,<int>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>
1,155,chromosome2,668344,790905,122561,ChrisC1,444778,567339
9,206,chromosome14,9398197,9591625,193428,ChrisC1,224256,417684
13,815,chromosome15,10015850,10580994,565144,ChrisC1,66814,631958
14,235,chromosome16,11462227,11683028,220801,ChrisC1,429498,650299
4,2603,chromosome5,2944974,3258307,313333,ChrisC3,65081,378414
11,901,chromosome11,6549018,6720267,171249,ChrisC3,18001,189250


In [5]:
head(GFF)

,seqid,source,type,start,end,score,strand,phase,attributes
,<fct>,<fct>,<fct>,<int>,<int>,<dbl>,<fct>,<fct>,<chr>
1,chromosome1,SGD,chromosome,1,230218,NA,NA,NA,ID=chromosome1;dbxref=NCBI:BK006935.2;Name=chromosome1
2,chromosome1,SGD,telomere,1,801,NA,-,NA,ID=TEL01L;Name=TEL01L;Note=Telomeric%20region%20on%20the%20left%20arm%20of%20Chromosome%20I%3B%20composed%20of%20an%20X%20element%20core%20sequence%2C%20X%20element%20combinatorial%20repeats%2C%20and%20a%20short%20terminal%20stretch%20of%20telomeric%20repeats;display=Telomeric%20region%20on%20the%20left%20arm%20of%20Chromosome%20I;dbxref=SGD:S000028862;curie=SGD:S000028862
3,chromosome1,SGD,X_element,337,801,NA,-,NA,Parent=TEL01L;Name=TEL01L_X_element
4,chromosome1,SGD,X_element_combinatorial_repeat,63,336,NA,-,NA,Parent=TEL01L;Name=TEL01L_X_element_combinatorial_repeat
5,chromosome1,SGD,telomeric_repeat,1,62,NA,-,NA,Parent=TEL01L;Name=TEL01L_telomeric_repeat_1
6,chromosome1,SGD,telomere,1,801,NA,-,NA,ID=TEL01L_telomere;Name=TEL01L_telomere;Parent=TEL01L


In [6]:
GFF<-filter(GFF, type=="gene")

In [7]:
extractGO<-function(x){
    if(sum(grepl("Ontology_term", x, fixed=TRUE))==0){
        return(NULL)
    }
    x<-strsplit(x, ";")[[1]]
    GO<-x[which(startsWith(x, "Ontology_term"))]
    GO<-str_replace(GO, fixed("Ontology_term="), "")
    GO<-strsplit(GO, ",")[[1]]
    
    return(GO)
}

background_GO<-unlist(lapply(pull(GFF, attributes), extractGO))
background_GO<-as.data.frame(table(background_GO))%>%arrange(-Freq)
colnames(background_GO)[1]<-"GO"
background_GO<-as.data.table(background_GO)
background_GO<-filter(background_GO, substr(GO, 1,2)=="GO")
head(background_GO)

GO,Freq
<fct>,<int>
GO:0003674,2519
GO:0005739,2471
GO:0005737,1916
GO:0005634,1882
GO:0008150,1746
GO:0005575,1273


In [ ]:
for(j in 1:nrow(TRD_regions)){
    
    TRD_regions_slice<-slice(TRD_regions, j)
    
    if(file.exists(paste0(output_dir,"/",pull(TRD_regions_slice, cross), ".", pull(TRD_regions_slice, ID), ".reducedTerms.RDS"))){
        if(file.mtime(paste0(output_dir,"/",pull(TRD_regions_slice, cross), ".", pull(TRD_regions_slice, ID), ".reducedTerms.RDS"))>max(file.mtime(list.files("/home/jnrunge/data/TRD/results/shiny/",pattern="TRD_regions.csv.gz$",full.names = TRUE)))){
            next
        }
    }
    
    

    GFF_subset<-filter(GFF, seqid==pull(TRD_regions_slice, chr),
                      (start<=pull(TRD_regions_slice,start) & end>=pull(TRD_regions_slice,start)) |
                       (start<=pull(TRD_regions_slice,end) & end>=pull(TRD_regions_slice,end)) |
                       (start<=pull(TRD_regions_slice,start) & end>=pull(TRD_regions_slice,end)) |
                       (start>=pull(TRD_regions_slice,start) & end<=pull(TRD_regions_slice,end)))



    GO_terms<-unlist(lapply(pull(GFF_subset, attributes), extractGO))
    
    GO_terms<-as.data.frame(table(GO_terms))%>%arrange(-Freq)
    if(nrow(GO_terms)<1){
        next
    }
    
    colnames(GO_terms)[1]<-"GO"
    colnames(GO_terms)[2]<-"Freq_location"
    GO_terms<-as.data.table(GO_terms)
    GO_terms<-filter(GO_terms, substr(GO, 1,2)=="GO")

    not_in_GO_terms<-filter(background_GO, !(GO %in% pull(GO_terms, GO)))%>%pull(GO)
    GO_terms<-bind_rows(GO_terms, data.table(GO=not_in_GO_terms, Freq_location=0))



    GO_terms<-full_join(GO_terms, background_GO, by="GO")

    GO_terms<-mutate(GO_terms, Other_location=sum(Freq_location)-Freq_location, Other=sum(Freq)-Freq)

    head(GO_terms)

    GO_terms<-filter(GO_terms,(Freq_location/Other_location) > (Freq/Other))

    nrow(GO_terms)

    GO_terms<-mutate(GO_terms, pChisq=NA)
    for(i in 1:nrow(GO_terms)){
        col1<-slice(GO_terms, i)%>%select(Freq_location,Other_location)%>%t()
        col2<-slice(GO_terms, i)%>%select(Freq,Other)%>%t()
        colnames(col1)<-"col1"
        colnames(col2)<-"col2"
        cols<-bind_cols(col1,col2)
        p_chi<-chisq.test(cols, simulate.p.value = TRUE, B = round((1/(0.05/nrow(GO_terms)))*2))$p.value # 2x as many replicates as would be needed to reach corrected significance if possible
        GO_terms<-mutate(GO_terms, pChisq=ifelse(row_number()==i,
                                                p_chi,
                                                pChisq))
    }

    nrow(filter(GO_terms, (Freq_location/Other_location) > (Freq/Other)))

    -log10(0.05/nrow(GO_terms))

    TRD_regions_slice

    p<-ggplot(GO_terms, aes(-log10(pChisq)))+
    geom_histogram(binwidth=0.1)+theme_bw(20)+ggtitle("Chisq GO terms vs global of over-represented GO terms")+
    geom_vline(xintercept=-log10(0.05/nrow(GO_terms)), color="red")

    saveRDS(p, file=paste0(output_dir,"/",pull(TRD_regions_slice, cross), ".", pull(TRD_regions_slice, ID), ".GOpdist.RDS"))


    saveRDS(arrange(GO_terms, pChisq), file=paste0(output_dir,"/",pull(TRD_regions_slice, cross), ".", pull(TRD_regions_slice, ID), ".GOdata.RDS"))

    
    GO_terms_enriched<-filter(GO_terms, pChisq<(0.05/nrow(GO_terms)))


    simMatrix <- calculateSimMatrix(pull(GO_terms_enriched, GO),
                                    orgdb="org.Sc.sgd.db", # yeast Sc 
                                    ont="BP", # biological processes; could also use CC or MF
                                    method="Rel")
    
    if(is.na(simMatrix)){
        next
    }
    if(length(simMatrix)==1){
        next
    }

    scores <- setNames(-log10(pull(GO_terms_enriched, pChisq)), pull(GO_terms_enriched, GO))
    reducedTerms <- reduceSimMatrix(simMatrix,
                                    scores,
                                    threshold=0.7,
                                    orgdb="org.Sc.sgd.db")

    saveRDS(reducedTerms, file=paste0(output_dir,"/",pull(TRD_regions_slice, cross), ".", pull(TRD_regions_slice, ID), ".reducedTerms.RDS"))
}

preparing gene to GO mapping data...

preparing IC data...

Warning message in calculateSimMatrix(pull(GO_terms_enriched, GO), orgdb = "org.Sc.sgd.db", :
"Removed 2 terms that were not found in orgdb for BP"
preparing gene to GO mapping data...

preparing IC data...

Warning message in calculateSimMatrix(pull(GO_terms_enriched, GO), orgdb = "org.Sc.sgd.db", :
"Removed 5 terms that were not found in orgdb for BP"
Warning message in if (is.na(simMatrix)) {:
"the condition has length > 1 and only the first element will be used"
preparing gene to GO mapping data...

preparing IC data...

Warning message in calculateSimMatrix(pull(GO_terms_enriched, GO), orgdb = "org.Sc.sgd.db", :
"Removed 2 terms that were not found in orgdb for BP"
Warning message in if (is.na(simMatrix)) {:
"the condition has length > 1 and only the first element will be used"
preparing gene to GO mapping data...

preparing IC data...

Warning message in calculateSimMatrix(pull(GO_terms_enriched, GO), orgdb = "org.Sc.sgd

In [ ]:

treemapPlot(readRDS(paste0(output_dir,"/",pull(TRD_regions_slice, cross), ".", pull(TRD_regions_slice, ID), ".reducedTerms.RDS")))

In [ ]:
wordcloudPlot(readRDS(paste0(output_dir,"/",pull(TRD_regions_slice, cross), ".", pull(TRD_regions_slice, ID), ".reducedTerms.RDS")), min.freq=1, colors="black")